## Data Preprocessing

### Nearest Grid Weather Station
Find the nearest grid weather station to each air statioin based on the geo_location.

In [1]:
import pandas as pd
import numpy as np

def euclidean_distance(x,y):
    return np.linalg.norm(x-y)

df_air_info = pd.read_csv('Air_Station_ID.csv')
df_grid_info = pd.read_csv('Beijing_grid_weather_station.csv')

no_air = df_air_info['stationId'].count()
no_grid = df_grid_info['gridId'].count()

air_join_grid = []
for i in range(no_air):
    temp = []
    air_geo = np.array([df_air_info['long'].iloc[i],df_air_info['lat'].iloc[i]])
    for j in range(no_grid):
        grid_geo = np.array([df_grid_info['long'].iloc[j],df_grid_info['lat'].iloc[j]])
        distance = euclidean_distance(air_geo,grid_geo)
        temp.append([df_grid_info['gridId'].iloc[j],distance])
    nearest_index = np.argmin(np.array(temp),axis = 0)[1]
    air_join_grid.append([df_air_info['stationId'].iloc[i], df_grid_info['gridId'].iloc[nearest_index]])

In [2]:
air_join_grid

[['dongsi_aq', 'beijing_grid_303'],
 ['tiantan_aq', 'beijing_grid_303'],
 ['guanyuan_aq', 'beijing_grid_282'],
 ['wanshouxigong_aq', 'beijing_grid_303'],
 ['aotizhongxin_aq', 'beijing_grid_304'],
 ['nongzhanguan_aq', 'beijing_grid_324'],
 ['wanliu_aq', 'beijing_grid_283'],
 ['beibuxinqu_aq', 'beijing_grid_263'],
 ['zhiwuyuan_aq', 'beijing_grid_262'],
 ['fengtaihuayuan_aq', 'beijing_grid_282'],
 ['yungang_aq', 'beijing_grid_239'],
 ['gucheng_aq', 'beijing_grid_261'],
 ['fangshan_aq', 'beijing_grid_238'],
 ['daxing_aq', 'beijing_grid_301'],
 ['yizhuang_aq', 'beijing_grid_323'],
 ['tongzhou_aq', 'beijing_grid_366'],
 ['shunyi_aq', 'beijing_grid_368'],
 ['pingchang_aq', 'beijing_grid_264'],
 ['mentougou_aq', 'beijing_grid_240'],
 ['pinggu_aq', 'beijing_grid_452'],
 ['huairou_aq', 'beijing_grid_349'],
 ['miyun_aq', 'beijing_grid_392'],
 ['yanqin_aq', 'beijing_grid_225'],
 ['dingling_aq', 'beijing_grid_265'],
 ['badaling_aq', 'beijing_grid_224'],
 ['miyunshuiku_aq', 'beijing_grid_414'],
 ['d

### Clustering of Air Station

In [2]:
from sklearn.cluster import KMeans
from sklearn import metrics
import numpy as np
    
X = df_air_info[['long','lat']].values

#cluster = DBSCAN(eps=0.24, min_samples=2).fit(X)
cluster_result = []
for n in range(2,10):
    score = 0
    for k in range(10):
        cluster = KMeans(n_clusters=n, random_state=k).fit(X)
        cluster_label = cluster.labels_
        score += metrics.silhouette_score(X, cluster_label, metric='euclidean')/10
    print("No.of clusters : %s"  %n + " , silhouette_score : %.3f" %score)
    cluster_result.append(score)
    
#Optimal No. of cluster = 6
cluster = KMeans(n_clusters=6, random_state=10).fit(X)
cluster_label = cluster.labels_
clusters = []
for i in set(cluster_label):
    temp = [x[0] for j,x in enumerate(df_air_info.values) if cluster_label[j] == i]
    clusters.append(temp)

No.of clusters : 2 , silhouette_score : 0.497
No.of clusters : 3 , silhouette_score : 0.477
No.of clusters : 4 , silhouette_score : 0.410
No.of clusters : 5 , silhouette_score : 0.418
No.of clusters : 6 , silhouette_score : 0.442
No.of clusters : 7 , silhouette_score : 0.429
No.of clusters : 8 , silhouette_score : 0.437
No.of clusters : 9 , silhouette_score : 0.449


### Based on the cluster result to fill nan element

In [8]:
df_air_1804 = pd.read_csv('aiqQuality_201804.csv')
df_air_1802_1803 = pd.read_csv('airQuality_201802-201803.csv')
df_air_1701_1801 = pd.read_csv('airQuality_201701-201801.csv')

df_air_1804.rename(index=str, columns={"station_id" : "stationId"
                                       ,"PM25_Concentration": "PM2.5"
                                       ,"PM10_Concentration": "PM10"
                                      ,"NO2_Concentration" : "NO2"
                                      ,"CO_Concentration" : "CO"
                                      ,"O3_Concentration" : "O3"
                                      ,"SO2_Concentration" : "SO2"}, inplace = True)

df_air_1802_1803.rename(index=str, columns={"utc_time" : "time"}, inplace = True)
df_air_1701_1801.rename(index=str, columns={"utc_time" : "time"}, inplace = True)

df_air_1804.drop(['id'], axis = 1, inplace = True)


In [9]:
from datetime import datetime , timedelta
import numpy as np
import math

# Here loading the file for each dataframe
# df_air = df_air_1802_1803
# df_air = df_air_1701_1801
df_air = df_air_1804
df_air['time'] = pd.to_datetime(df_air['time'])
df_air['time'] = df_air['time'].astype(str)

datetime_str_start = df_air['time'].min()
datetime_str_end = df_air['time'].max()

dt_start = datetime.strptime(datetime_str_start, "%Y-%m-%d %H:%M:%S")
dt_end = datetime.strptime(datetime_str_end, "%Y-%m-%d %H:%M:%S")

time_length = int((dt_end - dt_start)/timedelta(hours = 1))

for i in range(time_length + 1):
    dt_pred = dt_start + timedelta(hours = i)
    df_temp = df_air[(df_air['time'] == dt_pred.strftime("%Y-%m-%d %H:%M:%S"))]
    #fillna by their corresponding cluster
    for k,cluster_list in enumerate(clusters):
        df_temp_cluster = df_temp[df_temp['stationId'].isin(cluster_list)]
        PM25_mean = df_temp_cluster['PM2.5'].mean()
        PM10_mean = df_temp_cluster['PM10'].mean()
        O3_mean = df_temp_cluster['O3'].mean()
        #Error Detection for NaN value
        if math.isnan(PM25_mean) == False:
            df_temp_cluster['PM2.5'] = df_temp_cluster['PM2.5'].fillna(int(PM25_mean))
        if math.isnan(PM10_mean) == False:
            df_temp_cluster['PM10'] = df_temp_cluster['PM10'].fillna(int(PM10_mean))
        if math.isnan(O3_mean) == False:
            df_temp_cluster['O3'] = df_temp_cluster['O3'].fillna(int(O3_mean))
        
        if k == 0:
            df_temp_fill = df_temp_cluster
        else:
            df_temp_fill = df_temp_fill.append(df_temp_cluster)
            
    PM25_mean = df_temp_fill['PM2.5'].mean()
    PM10_mean = df_temp_fill['PM10'].mean()
    O3_mean = df_temp_fill['O3'].mean()
    #Error Detection for NaN value
    if math.isnan(PM25_mean) == False:
        df_temp_fill['PM2.5'] = df_temp_fill['PM2.5'].fillna(int(PM25_mean))
    if math.isnan(PM10_mean) == False:
        df_temp_fill['PM10'] = df_temp_fill['PM10'].fillna(int(PM10_mean))
    if math.isnan(O3_mean) == False:
        df_temp_fill['O3'] = df_temp_fill['O3'].fillna(int(O3_mean))
    
    if i == 0:
        df_air_temp = df_temp_fill
    else:
        df_air_temp = df_air_temp.append(df_temp_fill)
        
df_air_temp = df_air_temp.sort_values(by=['stationId'])
df_air_temp = df_air_temp.sort_values(by=['time'])
df_air_temp = df_air_temp.fillna(method = 'ffill')
df_air_temp = df_air_temp.fillna(method = 'bfill')

#Save the file in csv format
df_air_temp.to_csv('aiqQuality_201804(fillna).csv', index = False)

C:\Users\Juliana\Anaconda2\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Juliana\Anaconda2\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Juliana\Anaconda2\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pan

### Use the neraset grid station to encode the weather attribute of the air station

In [3]:
import pandas as pd
import os

#loading of every filled csv of air quality data
df_air_1804 = pd.read_csv('aiqQuality_201804(fillna).csv')
df_air_1802_1803 = pd.read_csv('airQuality_201802-201803(fillna).csv')
df_air_1701_1801 = pd.read_csv('airQuality_201701-201801(fillna).csv')
df_grid_1804 = pd.read_csv('gridWeather_201804.csv')
df_grid_1701_1803 = pd.read_csv('gridWeather_201701-201803.csv')
station_id = df_air_1804['stationId'].unique()

df_grid_1804.drop(['id','weather'], axis = 1, inplace = True)
df_grid_1701_1803.drop(['longitude', 'latitude'], axis = 1, inplace = True)

df_grid_1701_1803.rename(index=str, columns={"stationName" : "station_id"
                                             ,"utc_time" : "time"
                                             ,"wind_speed/kph" : "wind_speed"}, inplace = True)

storing_path = "C:/Users/user/Desktop/MSBD5002 Forecast Project/station_data/"
if not os.path.exists(storing_path):
    os.makedirs(storing_path)
    
for k in range(len(air_join_grid)):
    df_1804 = df_air_1804[df_air_1804['stationId'] == air_join_grid[k][0]]
    df_1802_1803 = df_air_1802_1803[df_air_1802_1803['stationId'] == air_join_grid[k][0]]
    df_1701_1801 = df_air_1701_1801[df_air_1701_1801['stationId'] == air_join_grid[k][0]]
    df_g_1701_1803 = df_grid_1701_1803[df_grid_1701_1803["station_id"] == air_join_grid[k][1]]
    df_g_1804 = df_grid_1804[df_grid_1804["station_id"] == air_join_grid[k][1]]

    frames_air = [df_1701_1801, df_1802_1803, df_1804]
    frames_grid = [df_g_1701_1803, df_g_1804]
    df_air = pd.concat(frames_air)
    df_grid = pd.concat(frames_grid)
    result = pd.merge(df_air, df_grid, how='left', on=['time'])
    result.drop(['station_id','NO2','CO','SO2'], axis = 1, inplace = True)
    result.dropna(inplace = True)
    del df_air
    del df_grid
    result.to_csv(os.path.join(storing_path,air_join_grid[k][0] + '.csv'), index = False)